# Win Probability based on the objectives taken by professional League of Legends teams
This report will outline my mini project for the class "Machine Learning for Media-Experiences" at Aalborg University Copenhagen for the Medialogy Masters class. In this mini project I will be looking at the professional scene of the MOBA game "League of Legends", and more specifically the different neutral objectives and how much they affect the killing team chances of winning.

## Background and details about the objectives in the game
In the game of "League of Legends" there are many factors that influence which team wins, and the ones I choose to point out are the neutral objectives, i.e. objectives that both teams fight over for an increase in power. One of the objectives are the Elemental Dragons, which reside in the bottom half of the map. These elemental dragons give the killing team a permanent buff in a specific stat, depending on the drake.

# Things to look at
Class applications something to remove bias
Principal component a... (TSNE)
optimizers
every factor relating to it

In [21]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets

In [22]:
df = pd.read_csv("data/2023_LoL_esports_match_data_from_OraclesElixir.csv")

# Usually you would want to remove the data point if they are empty, however in this case I wanna do the opposite
# I remove all the values that have a player, as I am only interested in the teams performance, not the individual.
df = df[df['playername'].isna()]

df.head()

C:\Users\lukas\AppData\Local\Temp\ipykernel_29356\2097972276.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/2023_LoL_esports_match_data_from_OraclesElixir.csv")


,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
10,ESPORTSTMNT06_2753012,complete,NaN,LFL2,2023,Spring,0,2023-01-10 17:07:16,1,13.01,...,535.0,-530.0,-1671.0,-37.0,0.0,0.0,1.0,1.0,1.0,0.0
11,ESPORTSTMNT06_2753012,complete,NaN,LFL2,2023,Spring,0,2023-01-10 17:07:16,1,13.01,...,498.0,530.0,1671.0,37.0,1.0,1.0,0.0,0.0,0.0,1.0
22,ESPORTSTMNT06_2754023,complete,NaN,LFL2,2023,Spring,0,2023-01-10 18:15:01,1,13.01,...,532.0,673.0,530.0,-34.0,4.0,6.0,3.0,3.0,4.0,4.0
23,ESPORTSTMNT06_2754023,complete,NaN,LFL2,2023,Spring,0,2023-01-10 18:15:01,1,13.01,...,498.0,-673.0,-530.0,34.0,3.0,4.0,4.0,4.0,6.0,3.0
34,ESPORTSTMNT06_2755035,complete,NaN,LFL2,2023,Spring,0,2023-01-10 19:20:51,1,13.01,...,452.0,-1901.0,-763.0,58.0,2.0,1.0,4.0,4.0,6.0,2.0


In [23]:
# Get information about the dataframe
df.drop(df.columns[[0]], axis=1, inplace=True)
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 21727 entries, 10 to 130175
Columns: 122 entries, datacompleteness to opp_deathsat15
dtypes: float64(85), int64(20), object(17)
memory usage: 20.4+ MB


In [24]:
# Look at the general values of the data. Note that popularity can be '0' - and there are many rows with that value
df.describe()


,year,playoffs,game,patch,participantid,gamelength,result,kills,deaths,assists,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
count,21727.000000,21727.000000,21727.000000,21707.000000,21727.000000,21727.000000,21727.000000,21727.000000,21727.000000,21727.000000,...,18474.000000,18474.000000,18474.000000,18474.000000,18474.000000,18474.000000,18474.000000,18474.000000,18474.000000,18474.000000
mean,2023.035164,0.208865,1.649514,13.084311,149.795324,1877.932756,0.500023,14.105031,14.137018,32.552584,...,505.955830,-0.190809,-0.240338,0.002111,3.989932,6.583306,4.003735,3.990527,6.582115,4.003951
std,0.184197,0.406507,0.939975,0.055155,50.258132,334.788793,0.500012,7.410860,7.409297,18.509019,...,38.993354,3053.806893,1900.764890,39.741334,2.854813,5.258892,2.858874,2.854388,5.259600,2.858816
min,2023.000000,0.000000,1.000000,13.010000,1.000000,976.000000,0.000000,0.000000,0.000000,0.000000,...,6.000000,-17056.000000,-9580.000000,-202.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2023.000000,0.000000,1.000000,13.040000,100.000000,1644.000000,0.000000,8.000000,8.000000,17.000000,...,484.000000,-1896.000000,-1187.000000,-26.000000,2.000000,3.000000,2.000000,2.000000,3.000000,2.000000
50%,2023.000000,0.000000,1.000000,13.100000,100.000000,1836.000000,1.000000,14.000000,14.000000,32.000000,...,509.000000,-1.000000,-1.000000,0.000000,3.000000,5.000000,3.000000,3.000000,5.000000,3.000000
75%,2023.000000,0.000000,2.000000,13.130000,200.000000,2074.000000,1.000000,19.000000,19.000000,46.000000,...,531.000000,1896.000000,1187.000000,26.000000,5.000000,9.000000,5.000000,5.000000,9.000000,5.000000
max,2024.000000,1.000000,5.000000,13.220000,200.000000,3604.000000,1.000000,49.000000,50.000000,117.000000,...,611.000000,17056.000000,9580.000000,202.000000,26.000000,37.000000,26.000000,26.000000,37.000000,26.000000


In [25]:
#import wandb